In [11]:
#Import libraries

import numpy as np
import pandas as pd
from hmmlearn import hmm
import warnings
warnings.filterwarnings("ignore")

In [12]:
#Dataset

df_main = pd.read_csv('Data/cleaned.txt', sep=",")
print("Dataset size is",len(df_main))
print(df_main.head(5))
X = df_main.values

Dataset size is 11087
     cdRPKM0    cdRPKM1    cdRPKM2
0   8.539825   5.879642   5.990043
1  13.837680   4.128452   6.882499
2  39.456786  62.505536  84.519655
3   2.104887   3.488833   2.908098
4  10.010800  17.066902   9.484017


In [39]:
#Fit HMM

remodel = hmm.GaussianHMM(n_components=3, n_iter=100) #consider Gaussian Emissions
remodel.fit(X) #get sequence of states using Viterbi
Z = remodel.predict(X)

In [43]:
print("Hidden states are")
for idx,i in enumerate(Z):
    if(idx%1000==0):
        print("State ", idx, " is ",i)

Hidden states are
State  0  is  0
State  1000  is  1
State  2000  is  2
State  3000  is  0
State  4000  is  2
State  5000  is  0
State  6000  is  1
State  7000  is  2
State  8000  is  0
State  9000  is  2
State  10000  is  2
State  11000  is  0


In [44]:
print("Transition matrix")
print(remodel.transmat_)
print()

Transition matrix
[[ 0.54087634  0.07999367  0.37912999]
 [ 0.52177293  0.12048685  0.35774022]
 [ 0.53537772  0.08882211  0.37580018]]



In [45]:
print("Means and vars of each hidden state")
for i in range(remodel.n_components):
    print("Hidden state {0}".format(i+1))
    print("mean = ", remodel.means_[i])
    print("var = ", np.diag(remodel.covars_[i]))
    print()

Means and vars of each hidden state
Hidden state 1
mean =  [ 6.1462972   6.30836619  7.09602962]
var =  [ 18.49730443  19.10018912  22.95519797]

Hidden state 2
mean =  [ 205.09762894  224.30634609  182.8674107 ]
var =  [ 81400.05687636  90080.63250793  44741.89038295]

Hidden state 3
mean =  [ 29.51901029  32.28109089  32.0453337 ]
var =  [ 266.40594043  303.6431122   271.47757482]

